In [1]:
import dataset as dataset

train_loader, val_loader, test_loader = dataset.get_dataloaders(dataset_folder = '/home/jy1559/Mar2025_Module/Datasets',
                                            dataset_name='Globo',
                                              train_batch_th=10000,
                                              use_bucket_batching=True,
                                              use_add_info=True,)


dict_keys(['data'])


In [2]:
dataset_name = 'Globo'
if dataset_name == 'Globo':
    add_info_num_cat = [('cat', 11), ('cat', 4), ('cat', 4), ('cat', 20), ('cat', 7), ('cat', 28)]
elif dataset_name == 'LFM-BeyMS':
    add_info_num_cat = []
elif dataset_name == 'Retail_Rocket':
    add_info_num_cat = [('cat', 3), ('num', -1)]

In [3]:
print(next(iter(train_loader)).keys())

dict_keys(['item_id', 'interaction_mask', 'session_mask', 'add_info', 'delta_ts'])


In [4]:
import dataset as dataset
llm_embedding = dataset.get_llm_embedding(dataset_folder = '/home/jy1559/Mar2025_Module/Datasets',
                                            dataset_name='Globo',)

In [5]:
print(llm_embedding['embedding_tensor'].shape[0])

46034


In [6]:
num_items = llm_embedding['embedding_tensor'].shape[0]
input_embedding_config = {
    'dt_method': 'bucket',
    'num_buckets': 32,
    'bucket_size': 2,
    'use_add_info': True,
    'add_info_specs': add_info_num_cat,
    'dataset_folder' : '/home/jy1559/Mar2025_Module/Datasets',
    'dataset_name': 'Globo',
    'use_llm': True,
    'use_add_info': True,
    'device' : 'cuda:3',
}

In [7]:
import model_code.input_embedding as input_embedding
input_embedder = input_embedding.InputEmbedding(
    n_items=num_items, **input_embedding_config)
input_embedder = input_embedder.to('cuda:3')

ID Embedding: 46034 items, 64 dim
LLM Embedding: 46034 items, 250 dim


In [8]:
import model_code.model as model
model_config = {
    'dataset_name': 'Globo',
    'dt_method': 'bucket',
    'num_buckets': 32,
    'bucket_size': 2,
    'use_add_info': False,
    'use_llm': False,
    'use_dt': False,
    'num_layers': 2,
    'hidden_size': 512,
    'num_heads': 8,
    'dropout': 0.1,
    'add_info_specs': add_info_num_cat,
    'device': 'cuda:3',
}
mod = model.SeqRecModel(num_items, model_config).to('cuda:3')

ID Embedding: 46034 items, 64 dim


In [9]:
import torch
batch = next(iter(train_loader))
batch = {k: v.to("cuda:3") if torch.is_tensor(v) else v  for k, v in batch.items()}

In [ ]:
from time import time
from loss import compute_loss
from tqdm.auto import tqdm
import os, torch
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
torch.autograd.set_detect_anomaly(True)
i = 0
for batch in tqdm(train_loader, total=len(train_loader)):
    start_time = time()
    batch = {k: v.to("cuda:3") if torch.is_tensor(v) else v for k, v in batch.items()}
    out = mod(batch)
    loss = compute_loss(batch, out, mod, {'dataset_name': 'Globo'})
    end_time = time()
    if i % 500 == 0: print(f"Batch processed in {end_time - start_time:.4f} seconds, loss = {loss.item():.4f}")
    i += 1
    

  0%|          | 0/2706 [00:00<?, ?it/s]

Batch processed in 1.4297 seconds, loss = 5.8161


In [ ]:
batch = next(iter(train_loader))
batch = {k: v.to("cuda:3") if torch.is_tensor(v) else v  for k, v in batch.items()}
mod(batch)

TypeError: 'dict' object is not an iterator

In [ ]:
import pickle
with open('/home/jy1559/Mar2025_Module/Datasets/Globo/item_embedding_dense.pickle', 'rb') as f:
    item_embedding_dense = pickle.load(f)
with open('/home/jy1559/Mar2025_Module/Datasets/Globo/item_embedding_normalized_revised.pickle', 'rb') as f:
    item_embedding_revised = pickle.load(f)
print(item_embedding_dense['embedding_tensor'].shape)
print(item_embedding_revised['embedding_tensor'].shape)

torch.Size([46034, 384])
torch.Size([364047, 384])


In [ ]:
import model_code.session_encoder as session_encoder
session_encoder_model = session_encoder.SessionEncoder()
session_encoder_model = session_encoder_model.to('cuda:3')
usr_embedding = input_embedding[:, 0, 0, :]
sesseion_encoder_output, user_embedding_input, loss_mask = session_encoder_model(input_embedding[:, 0, :, :],
               batch['interaction_mask'][:, 0, :].to('cuda:3'),
               usr_embedding=usr_embedding,)

TypeError: 'module' object is not subscriptable

In [ ]:
import torch
from model_code.user_update import UserStateUpdater
B, D_sess = 4, 256
sess_vec   = torch.randn(B, D_sess).to('cuda:3')
delta_t    = torch.tensor([30., 3600., 180., 600.]).to('cuda:3')  # 초 단위

updater = UserStateUpdater(d_session=D_sess,
                           method='default',
                           rnn='GRU',
                           device='cuda:3',
                           dt_emb_module=input_embedder.dt_emb,)

state_0 = updater.reset_state(batch['delta_ts'].shape[0], 'cuda:3')
print(user_embedding_input.shape)  # [B, D_sess]
print(batch['delta_ts'].to('cuda:3').shape) # [B, S, I]
state_1 = updater(user_embedding_input, state_0, dt_sec=batch['delta_ts'][:, 0, 0].to('cuda:3'), pad_mask=batch['interaction_mask'][:, 0, 0].to('cuda:3'))

print(state_1.shape)   # GRU → [1,B,hidden]


torch.Size([6, 256])
torch.Size([6, 26, 8])
prev_state shape: torch.Size([1, 6, 256])
pad_mask) shape: torch.Size([6])
dt_sec shape: torch.Size([6])
dt_emb shape: torch.Size([6, 16])
session_vec shape: torch.Size([6, 256])
x_t shape: torch.Size([6, 1, 272])
x_t shape: torch.Size([6, 1, 272])
prev_state shape: torch.Size([1, 6, 256])
torch.Size([1, 6, 256])
